<img src="http://www.ceic.org.ar/spanish/img/siisa-logo.jpg" align='right'>

# Ses Model

Clasificacion del modelo SES utilizando  ensambles.

### Primero importo todas las librerias necesarias.

In [1]:
# Importando las librerias necesarias para trabajar correctamente.
#sql tools.
import pymssql
# basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from dask.diagnostics import ProgressBar
# inbalance learning.
from imblearn.over_sampling import ADASYN 
from imblearn.under_sampling import RandomUnderSampler
# metrics.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
# graph tree.
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
# tree methods.
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# siisa tools.
from siisa import *
# hide cell.
hide_toggle()

C:\Users\elias\Programs\anaconda3\envs\siisa\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


#### Query sobre MINISIISA para realizar el modelo:

In [2]:
query =""" select 
cob.cuil
, cob.age
, cob.IQ3M
, cob.IQ6M
, cob.IQ12M
, cob.IQNEW
, cob.IQOLD
, cob.C_WCURR
, cob.C_TOPEN
, cob.C_G1
, cob.C_G2
, cob.C_G39
, cob.C_G69
, cob.C_G79
, cob.C_G99
, cob.C_6G1
, cob.C_6G2
, cob.C_6G39
, cob.C_6G69
, cob.C_6G79
, cob.C_6G99
, cob.C_12G1
, cob.C_12G2
, cob.C_12G39
, cob.C_12G69
, cob.C_12G79
, cob.C_12G99
, cob.C_24G1
, cob.C_24G2
, cob.C_24G39
, cob.C_24G69
, cob.C_24G79
, cob.C_24G99
, cob.C_60G1
, cob.C_60G2
, cob.C_60G39
, cob.C_60G69
, cob.C_60G79
, cob.C_60G99
, cob.B_AVGMO
, cob.B_NEW
, cob.B_OLD
, cob.B_G1
, cob.B_G2
, cob.B_G3
, cob.B_G4
, cob.B_G0
, cob.B_6G1
, cob.B_6G2
, cob.B_6G3
, cob.B_6G4
, cob.B_6G0
, cob.B_12G1
, cob.B_12G2
, cob.B_12G3
, cob.B_12G4
, cob.B_12G0
, cob.B_24G1
, cob.B_24G2
, cob.B_24G3
, cob.B_24G4
, cob.B_24G0
, cob.SEXO
, cob.CH_RECH_30
, cob.CH_RECH_60
, cob.CH_RECH_90
, cob.CH_RECH_120
, cob.CH_RECH_180
, cob.AUTONOMO
, cob.RELAC_DEP
, cob.JUBILADO
, cob.EMPLE_PUB
, cob.ESTADO_CIVIL
, cob.ANTIGUEDAD_LAB
, cob.NACIONALIDAD
--, cob.Ingresos -- Borro campo porque es lo que quiero predecir.
, cob.MORA_FAMILIAR
, cob.BAJASXLEY
, cob.CAPITALTOTALMORA
, cob.PASOXABOGADO
, cob.maxDuracionMora
, cob.cantSalidasMora
, cob.tendenciaLineas6m
, cob.tendenciaLineas6m_r_sq
, cob.tendenciaLineas12m
, cob.tendenciaLineas12m_r_sq
, cob.tendenciaLineas24m
, cob.tendenciaLineas24m_r_sq
--, cob.tendenciaSueldo  -- en principio sacarla.
--, cob.tendenciaSueldo_r_sq  
, cob.tendenciaCant1s6m
, cob.tendenciaCant1s6m_r_sq
, cob.tendenciaCant1s12m
, cob.tendenciaCant1s12m_r_sq
, cob.tendenciaCant1s24m
, cob.tendenciaCant1s24m_r_sq
, cob.tendenciaMonto1s6m
, cob.tendenciaMonto1s6m_r_sq
, cob.tendenciaMonto1s12m
, cob.tendenciaMonto1s12m_r_sq
, cob.tendenciaMonto1s24m
, cob.tendenciaMonto1s24m_r_sq
, cob.cant_lab
, cob.cantGestiones
, cob.bacActivas
, cob.bacActivasMonto
, cob.zapasActivas
, cob.zapasActivasMonto
, cob.zapasAnteriores
, cob.compromiso
, cob.monto1
, cob.cant1
, cob.monto2
, cob.cant2
--, cob.ingresosSinCompromiso
, cob.subsector_pc
, cob.maxBancarizacion
, cob.mesesBancarizacion
, cob.cantBancos
, cob.tieneTelFijo
, cob.tieneCelLinea
, cob.tieneAuto
, cob.modeloAuto
--, cob.fecha              -- saco todas estas columnas que no me interesan.
--, cob.sesModelo
--, cob.sesCategoria
--, cob.idCat
--, cob.scorecard
--, cob.mora_historica
--, cob.mora_actual
--, cob.geo_sesModelo
--, cob.geo_sesCategoria
--, cob.geo_idCat
--, cob.idcat8
, cob.idcat10  --etiqueta a predicir.
from cobranzas.dbo.ses10 cob
"""
# creo dataframe.
#df=get_data_fsql(query,serverName='MINISIISA')
#df.to_csv('ses_data.csv')
# Cargo la data desde el file o me traigo de nuevo la data en la celda de arriba.
df= pd.read_csv('ses_data.csv')
hide_toggle()

In [3]:
df.shape

(203236, 121)

In [4]:
# Miro todas las columnas.
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_rows', 122)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,203236.00,101617.50,58669.32,0.00,50808.75,101617.50,152426.25,203235.00
cuil,203236.00,22921015516.60,3175778589.34,20009523058.00,20257014983.25,20379844015.50,27111051848.00,27955665932.00
age,203195.00,44.29,15.97,0.00,32.00,42.00,55.00,115.00
IQ3M,203236.00,0.79,1.39,0.00,0.00,0.00,1.00,51.00
IQ6M,203236.00,1.04,1.92,0.00,0.00,0.00,1.00,112.00
IQ12M,203236.00,1.40,2.68,0.00,0.00,0.00,2.00,113.00
IQNEW,203236.00,401.46,677.65,0.00,11.00,41.00,442.00,3249.00
IQOLD,203236.00,1286.80,1130.23,0.00,34.00,1207.00,2343.00,3382.00
C_WCURR,71519.00,1.89,1.49,1.00,1.00,1.00,3.00,11.00
C_TOPEN,203236.00,1.42,1.92,0.00,0.00,0.00,2.00,18.00


In [5]:
corr_df=df.corr()

In [6]:
# Miro la correlacion de las etiquetas con respecto a los features.
corr_df.idcat10

Unnamed: 0                  0.17
cuil                        0.15
age                         0.05
IQ3M                        0.16
IQ6M                        0.15
IQ12M                       0.14
IQNEW                      -0.15
IQOLD                      -0.03
C_WCURR                     0.16
C_TOPEN                    -0.02
C_G1                       -0.04
C_G2                        0.00
C_G39                       0.05
C_G69                       0.00
C_G79                        nan
C_G99                        nan
C_6G1                      -0.06
C_6G2                       0.00
C_6G39                      0.05
C_6G69                      0.00
C_6G79                       nan
C_6G99                       nan
C_12G1                     -0.04
C_12G2                      0.00
C_12G39                     0.05
C_12G69                     0.00
C_12G79                      nan
C_12G99                      nan
C_24G1                     -0.03
C_24G2                     -0.00
C_24G39   

In [7]:
# Evaluar las edades mayores que 100.
# Saco todos aquellos mayores de 100
df=df[~(df.age>100)]

In [8]:
# Impute con algun criterio que se discutio con Santi.
values = {'C_WCURR': 0, 'B_NEW': 9999, 'B_OLD': 9999, 'CAPITALTOTALMORA': 0, 'tendenciaMonto1s6m' :0, 'tendenciaMonto1s6m_r_sq' :0, 'tendenciaMonto1s12m':0,'tendenciaMonto1s12m_r_sq':0,'tendenciaMonto1s24m':0,'tendenciaMonto1s24m_r_sq':0, 'maxBancarizacion':0, 'mesesBancarizacion':0} 
df.fillna(value=values,inplace=True)

# Saco los valores de sexo que no se.(cat 0)
df= df[df.SEXO!=0]
# Saco los Nulls.
df= df[~df.SEXO.isna()]

In [9]:
#Arreglo la dummy de sexo.
sexo_dummies= pd.get_dummies(df.SEXO,prefix='SEXO')
# Renombro las columnas.
sexo_dummies.columns=['SEXO_M','SEXO_F']
# Agrego la dummy sexo.
df = pd.concat([df,sexo_dummies.SEXO_M ], axis=1)
# cambio los valores de subsector_pc por el valor promedio dentro de esa categoria.
subsector_pc_fix = df.groupby(['idcat10'])['subsector_pc'].transform(lambda x: x.fillna(x.mean()))
df['subsector_pc']=subsector_pc_fix


In [10]:
# miro lo que queda y la distribucion.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,202827.00,101583.32,58689.41,0.00,50744.50,101493.00,152442.50,203235.00
cuil,202827.00,22921304937.92,3176431539.13,20009523058.00,20256872345.50,20379411577.00,27111165374.00,27955665932.00
age,202827.00,44.30,15.83,0.00,32.00,42.00,55.00,100.00
IQ3M,202827.00,0.79,1.39,0.00,0.00,0.00,1.00,51.00
IQ6M,202827.00,1.04,1.93,0.00,0.00,0.00,1.00,112.00
IQ12M,202827.00,1.40,2.68,0.00,0.00,0.00,2.00,113.00
IQNEW,202827.00,401.92,677.90,0.00,11.00,41.00,443.00,3249.00
IQOLD,202827.00,1288.36,1130.33,0.00,35.00,1211.00,2344.00,3382.00
C_WCURR,202827.00,0.67,1.26,0.00,0.00,0.00,1.00,11.00
C_TOPEN,202827.00,1.42,1.92,0.00,0.00,0.00,2.00,18.00


In [11]:
# Evaluo los porcentages de cada clase antes de hacer cualquier calculo.
df.idcat10.value_counts()/len(df)*100

10   21.08
9    16.02
8    13.76
7    11.65
6     9.20
4     7.51
5     6.58
1     5.96
3     5.48
2     2.77
Name: idcat10, dtype: float64

In [12]:
#reparto en train test
# vector objetivo.
# Encodear las etiquetas usando LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['idcat10'])
# vector de features
X=df.copy()
# Borro algunas variables.
del X['idcat10']
del X['cuil']
del X['Unnamed: 0']
del X['AUTONOMO']
del X['RELAC_DEP']
del X['JUBILADO']
del X['EMPLE_PUB']
del X['ANTIGUEDAD_LAB']
del X['cant_lab']
del X['subsector_pc']
# Borro la columna de sexo.
del X['SEXO']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42,stratify=y)

In [13]:
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [14]:
pd.Series(y_train_rus).value_counts()/len(y_train_rus)*100

9   10.00
8   10.00
7   10.00
6   10.00
5   10.00
4   10.00
3   10.00
2   10.00
1   10.00
0   10.00
dtype: float64

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape,X_train_rus.shape, y_train_rus.shape


((182544, 111), (20283, 111), (182544,), (20283,), (50530, 111), (50530,))

 ## Me fijo si da mejor todos los datos o Undersampling.

In [16]:
# Using Undersampling.
X2_train = X_train_rus
Y2_train = y_train_rus
# Using all data.
#X2_train = X_train
#Y2_train = y_train

In [17]:
le.classes_

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [18]:

SEED = 1
treeclf = DecisionTreeClassifier(random_state=SEED)

# Import GridSearchCV

# Define the grid of hyperparameters 'params_dt'
params_dt = {
    'max_depth': list(range(3, 60, 1)),
    #'min_samples_leaf': [0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5],
    #'max_features': [0.2, 0.4, 0.6, 0.8]
    }
# Instantiate a 5-fold CV grid search object 'grid_dt'
grid_search = GridSearchCV(estimator=treeclf,param_grid=params_dt,scoring='f1_micro',cv=5,n_jobs=-1,verbose=1)
# Fit 'grid_dt' to the training data

with ProgressBar():
    grid_search.fit(X2_train, Y2_train)

# Extract best hyperparameters from 'grid_dt'
best_hyperparams = grid_search.best_params_
print('Best hyperparameters:\n', best_hyperparams)
# Extract best CV score from 'grid_dt'
best_CV_score = grid_search.best_score_
print('Best CV accuracy'.format(best_CV_score))

Fitting 5 folds for each of 57 candidates, totalling 285 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
best_tree=grid_search.best_estimator_
best_tree.fit(X2_train, Y2_train)

In [ ]:

atributos = X_test.columns
# Visualizar el árbol usando graphviz
dot_data = export_graphviz(best_tree, out_file=None,  
                feature_names=atributos,  
                filled=True, rounded=True,  
                special_characters=True)  
#graph = pydotplus.graph_from_dot_data(dot_data)
#Image(graph.create_png())

In [ ]:
best_tree.tree_.max_depth

In [ ]:
# Calcular la importancia de los atributos
pd.DataFrame({'Atributo':atributos,
              'importancia':best_tree.feature_importances_}).sort_values('importancia',
                                                                      ascending=False).head()

In [ ]:
y_pred=best_tree.predict(X_test)

In [ ]:

accuracy_score(y_test, y_pred)

In [ ]:
# Calcular y mostrar la matriz de confusión

conf = confusion_matrix(y_pred,y_test)
predicted_cols = ['pred_'+ str(c) for c in le.classes_]
#cm_rus=pd.DataFrame(conf, index = le.classes_, columns = predicted_cols)
cm_all=pd.DataFrame(conf, index = le.classes_, columns = predicted_cols)

In [ ]:

print(classification_report(y_test, y_pred, ))

In [ ]:
accuracyPlusOne(cm_all)

In [ ]:
cm_all

In [24]:
from xgboost.sklearn import XGBClassifier
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import time
import numpy as np
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
model_xg = XGBClassifier(n_jobs=-1)
one_to_left = st.beta(10, 1) # Esta distribución nos dará valores entre 0 y 1 mayormente cercanos a 1
params = {  
    "n_estimators": st.randint(20,40), # Number of boosted trees to fit.
    "max_depth": st.randint(3, 12),     # Maximum tree depth for base learners.
    "learning_rate": st.uniform(0.05, 0.4), #     Boosting learning rate (xgb’s “eta”)
    "colsample_bytree": one_to_left, #     Subsample ratio of columns when constructing each tree.
    "subsample": one_to_left,     # Subsample ratio of the training instance.
    "gamma": st.uniform(0, 10), #     Minimum loss reduction required to make a further partition on a leaf node of the tree.
    'reg_alpha': st.uniform(0.05,10),   # L1 regularization term on weights
    "min_child_weight": st.uniform(1,20), #    Minimum sum of instance weight(hessian) needed in a child.
}
xgb = RandomizedSearchCV(model_xg,params, n_iter=25, verbose= True)
start = time.time()
xgb.fit(X2_train, Y2_train)
end = time.time()
print("La búsqueda con XGBoost llevó: ", end - start, " segundos")

C:\Users\elias\Programs\anaconda3\envs\siisa\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 38.1min finished


La búsqueda con XGBoost llevó:  2332.8547728061676  segundos


In [26]:
best_model=xgb.best_estimator_

In [28]:
y_predicted_xgb = best_model.predict(X_test.values)


In [29]:
# Calcular y mostrar la matriz de confusión

conf = confusion_matrix(y_predicted_xgb,y_test)
predicted_cols = ['pred_'+ str(c) for c in le.classes_]
#cm_rus=pd.DataFrame(conf, index = le.classes_, columns = predicted_cols)
cm_all=pd.DataFrame(conf, index = le.classes_, columns = predicted_cols)

In [30]:
cm_all

,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10
1,579,189,247,298,161,162,152,165,110,104
2,207,115,236,247,179,218,220,174,129,90
3,103,66,147,153,117,150,157,149,88,72
4,62,38,99,124,103,142,138,145,109,74
5,51,26,76,107,98,136,144,144,124,83
6,50,24,52,100,131,187,227,177,126,96
7,41,29,65,133,149,227,304,351,290,177
8,40,26,72,144,144,265,365,463,497,411
9,39,20,57,123,149,230,407,592,935,875
10,36,29,60,95,103,149,248,432,841,2293


In [31]:
accuracy_score(y_predicted_xgb,y_test)

0.2585909382241286

In [32]:
accuracyPlusOne(cm_all)

0.5249223487649756